In [1]:
import os

In [2]:
%pwd

'c:\\Users\\alvin\\Documents\\Project Data Science\\End-To-End-ML-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\alvin\\Documents\\Project Data Science\\End-To-End-ML-Project'

In [12]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path 
    model_name: str
    alpha: float
    l1_ratio: float
    target_col: str

In [13]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [14]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)    
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_col=schema.name
        ) 
        return model_trainer_config

In [15]:
import pandas as pd
import os
from mlProject import logger
from sklearn.linear_model import ElasticNet
import joblib

In [18]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        
        train_x = train_data.drop(self.config.target_col, axis=1)
        test_x = test_data.drop(self.config.target_col, axis=1)
        train_y = train_data[self.config.target_col]
        test_y = test_data[self.config.target_col]
        
        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x, train_y)
        
        os.makedirs(self.config.root_dir, exist_ok=True)
        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))
        

In [ ]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2025-04-25 19:45:25,700]: INFO:common:yaml file: config\config.yaml loaded successfully]
[2025-04-25 19:45:25,703]: INFO:common:yaml file: params.yaml loaded successfully]
[2025-04-25 19:45:25,706]: INFO:common:yaml file: schema.yaml loaded successfully]
[2025-04-25 19:45:25,708]: INFO:common:created directory at: artifacts]
